In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from utils import CFEVideoConf, image_resize

In /home/tanishk/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/tanishk/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/tanishk/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/tanishk/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/tanishk/.local/lib/python3.6/site-packages/matplotlib/mpl-d

In [2]:
nose_cascade = cv2.CascadeClassifier("./Train/third-party/Nose18x15.xml")
face_cascade = cv2.CascadeClassifier("./Train/third-party/haarcascade_frontalface_alt.xml")
eyes_cascade = cv2.CascadeClassifier("./Train/third-party/frontalEyes35x16.xml")

In [3]:
imgMustache = cv2.imread("./Train/mustache.png",-1)
imgGlasses = cv2.imread("./Train/glasses.png",-1)

In [4]:
orig_mask = imgMustache[:,:,3]
orig_mask_inv = cv2.bitwise_not(orig_mask)

orig_mask_glasses = imgGlasses[:,:,3]
orig_mask_glasses_inv = cv2.bitwise_not(orig_mask_glasses)

In [5]:
imgMustache = imgMustache[:,:,0:3]
origMustacheHeight, origMustacheWidth = imgMustache.shape[:2]

imgGlasses = imgGlasses[:,:,0:3]
origGlassesHeight, origGlassesWidth = imgGlasses.shape[:2]

In [8]:
video_capture = cv2.VideoCapture(0)

while True:
    
    ret, frame = video_capture.read()
    
    if ret == False :
        
        continue
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray,1.3,5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        nose = nose_cascade.detectMultiScale(roi_gray,1.3,6,minSize =(60,60))
        eyes = eyes_cascade.detectMultiScale(roi_gray,1.3,6, minSize =(60,60))
    
        for (nx,ny,nw,nh) in nose:
        
            cv2.rectangle(roi_color,(nx,ny),(nx+nw,ny+nh),(255,0,0),2)
            mustacheWidth =  nw
            mustacheHeight = mustacheWidth * origMustacheHeight / origMustacheWidth
            
            x1 = int(nx - (mustacheWidth/4))
            x2 = int(nx + nw + (mustacheWidth/4))
            y1 = int(ny + nh - (mustacheHeight/1.5))
            y2 = int(ny + nh + (mustacheHeight/2))
            
            if x1 < 0:
                x1 = 0
            if y1 < 0:
                y1 = 0
            if x2 > w:
                x2 = w
            if y2 > h:
                y2 = h
                
            mustacheWidth = int(x2 - x1)
            mustacheHeight = int(y2 - y1)
            
            mustache = cv2.resize(imgMustache, (mustacheWidth,mustacheHeight), interpolation = cv2.INTER_AREA)
            mask = cv2.resize(orig_mask, (mustacheWidth,mustacheHeight), interpolation = cv2.INTER_AREA)
            mask_inv = cv2.resize(orig_mask_inv, (mustacheWidth,mustacheHeight), interpolation = cv2.INTER_AREA)
 
            roi = roi_color[y1:y2, x1:x2]
            roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
        
            roi_fg = cv2.bitwise_and(mustache,mustache,mask = mask)
            dst = cv2.add(roi_bg,roi_fg)
            
            
            roi_color[y1:y2, x1:x2] = dst
            
    
            
            
            break
                
        for (ex,ey,ew,eh) in eyes:
            
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,0,0),2)
            glassesWidth =  ew
            glassesHeight = glassesWidth * origGlassesHeight/origGlassesWidth
            
            ex1 = int(ex - (glassesWidth/1.5))
            ex2 = int(ex + ew + (glassesWidth/2.0))
            ey1 = int(ey + eh - (glassesHeight+20))
            ey2 = int(ey + eh + (glassesHeight/3))
            
            if ex1 < 0:
                ex1 = 0
            if ey1 < 0:
                ey1 = 0
            if ex2 > w:
                ex2 = w
            if ey2 > h:
                ey2 = h
                
            glassesWidth = int(ex2 - ex1)
            glassesHeight = int(ey2 - ey1)
            
            
            glasses = cv2.resize(imgGlasses, (glassesWidth,glassesHeight), interpolation = cv2.INTER_AREA)
            emask = cv2.resize(orig_mask_glasses, (glassesWidth,glassesHeight), interpolation = cv2.INTER_AREA)
            emask_inv = cv2.resize(orig_mask_glasses_inv, (glassesWidth,glassesHeight), interpolation = cv2.INTER_AREA)
            
            eroi_gl = roi_color[ey1:ey2, ex1:ex2]
            eroi_bg = cv2.bitwise_and(eroi_gl,eroi_gl,mask = emask_inv)
            
            eroi_fg = cv2.bitwise_and(glasses, glasses, mask = emask)
            edst = cv2.add(eroi_bg,eroi_fg)
            
            roi_color[ey1:ey2, ex1:ex2] = edst
            
            
            break
        
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
video_capture.release()
cv2.destroyAllWindows()